In [1]:
import numpy as np
from ortools.algorithms import pywrapknapsack_solver


In [2]:
# generate input sequence
def gen_inputs(T, weight_low, weight_high, p_min, p_max):
    weights = []
    for dim in range(2):
        inner_weights=np.zeros(T)
        for i in range(T):
            tmp=int(np.random.uniform(low=weight_low, high=weight_high))
            while tmp==0:
                tmp=int(np.random.uniform(low=weight_low, high=weight_high))
            inner_weights[i] = tmp
        weights.append(list(inner_weights))
    
    dens = np.random.uniform(low=p_min, high=p_max, size=T)
    values = list(np.multiply(np.mean(weights, axis=0),dens).astype(int))
    # print(weights[:10])
    # print(dens[:10])
    # print(values[:10])
    return values, weights   

## Online Threshold Solver

In [4]:
def get_phi(ys, beta, p_min, capacities):
    y=np.mean(ys)
    if y < beta:
        return p_min
    elif beta <= y and ys[0] <= capacities[0] and ys[1] <= capacities[1]:
        return p_min * np.exp(y / beta - 1)
    else: 
        print("ERROR! y exceeds max capacity!")
    
def get_total_vals(values, x_ts):
    total_val = 0
    for i in range(len(x_ts)):
        total_val += values[i]*x_ts[i]
    return total_val

def get_total_cap(weights, x_ts):
    total_caps=[0,0]
    for i in range(len(x_ts)):
        total_caps[0] += weights[0][i]*x_ts[i]
        total_caps[1] += weights[1][i]*x_ts[i]

    return total_caps

In [5]:
def threshold_solver(values, weights, capacities, p_min, p_max):
    sigma=10**-10 # handle zero weight

    beta=1/(1+np.log(p_max/p_min))
    beta=beta*capacities[0]
    # print('beta', beta)
    dim=len(weights)
    T = len(values)
    ys_ts = np.zeros((dim, T))
    x_ts = np.zeros(T)
    # load = 0
    
    for t in range(T):
        item_value = values[t]
        
        item_weights=[weights[0][t], weights[1][t]]
        for i in range(len(item_weights)):
            if item_weights[i] == 0:
                item_weights[i] += sigma

        val_densities = [item_value/item_weights[0], item_value/item_weights[1]]

        # get p_t
        p_t = p_min
        if t > 0:
            p_t = get_phi([ys_ts[0][t-1], ys_ts[1][t-1]], beta, p_min, capacities)
        # print('t:', t, ' p_t:', p_t, ' val_den:', val_density)
        # accept
        if np.mean(val_densities) >= p_t and ys_ts[0][t-1] + item_weights[0] <= capacities[0] and ys_ts[1][t-1] + item_weights[1] <= capacities[0]:
            x_ts[t] = 1
        # deny
        else:
            x_ts[t] = 0
        ys_ts[0][t] = ys_ts[0][t-1] + item_weights[0]*x_ts[t]
        ys_ts[1][t] = ys_ts[1][t-1] + item_weights[1]*x_ts[t]
        # capacity is full, end loop
        if ys_ts[0][t] == capacities[0] or ys_ts[1][t] == capacities[1]:
            break;
    alpha=beta
    return ys_ts, x_ts, alpha

In [6]:
 def get_online_res(values, weights, capacities, p_min, p_max):
    # min_den=10**10
    # max_den=0
    # sigma=10**-10
    # for i in range(len(values)):
    #     tmp_den = values[i]/(weights[0][i])
    #     # print(tmp_den)
    #     min_den = min(min_den, tmp_den)
    #     max_den = max(max_den, tmp_den)
    # print(min_den)
    # print(max_den)

#     p_min=10
#     p_max= max_den+10

    ys_ts, x_ts, alpha = threshold_solver(values, weights, capacities, p_min, p_max)
    total_val = get_total_vals(values, x_ts)
    total_cap=get_total_cap(weights, x_ts)
    # print('total value:', total_val)
    # print('total weight:', total_cap)
    # print('alpha:', alpha)
    return total_val, total_cap, alpha, x_ts
    # print('y_ts:', y_ts)
    # print('x_ts:', x_ts)

In [7]:
# print(1/(1+np.log(p_max/p_min)))
# print(np.log(p_max/p_min))

## Offline Solver from OR-Tools

In [8]:
# MULTIDIMENSION_CBC_MIP_SOLVER
solver = pywrapknapsack_solver.KnapsackSolver(
    pywrapknapsack_solver.KnapsackSolver.
    KNAPSACK_MULTIDIMENSION_CBC_MIP_SOLVER, 'KnapsackExample')


In [9]:
def get_offline_res(values, weights, capacities):
    solver.Init(values, weights, capacities)
    computed_value = solver.Solve()
    packed_items = []
    packed_weights = []
    total_weight = [0,0]
    # print('Total value =', computed_value)
    for i in range(len(values)):
        if solver.BestSolutionContains(i):
            packed_items.append(i)
            packed_weights.append([weights[0][i], weights[1][i]])
            total_weight[0] += weights[0][i]
            total_weight[1] += weights[1][i]

    # print('Total weight:', total_weight)
    # print('Packed items:', packed_items)
    # print('Packed_weights:', packed_weights)
    return computed_value, total_weight, packed_items

## Infinitesimal Setting (Q3)

In [65]:
## infinitesimal setting
weight_highs=np.ones(10)
for i in range(len(weight_highs)):
    weight_highs[i] = 10 * 10**i
print(weight_highs)

cr_list_highs=[]
for weight_high in weight_highs:
    cr_list=[]
    for itrs in range(100):
        T=100

        # infinitesimal setting
        p_min= 1
        p_max= 100
        weight_low = 1
        # weight_high = 1e-5


        values, weights = gen_inputs(T=T, weight_low=weight_low, weight_high=weight_high, p_min=p_min, p_max=p_max)
        # capacities=[max(1, int(np.sum(weights)/2))]
        capacities=[10**10, 10**10]
        # p_max= 10
        # print('Capacities:', capacities)

        total_val, total_cap, alpha, x_ts = get_online_res(values, weights, capacities, p_min, p_max)
        packed_idx=[i for i, e in enumerate(x_ts) if e != 0]

        computed_value, total_weight, packed_items = get_offline_res(values, weights, capacities)
        # cr = total_cap/total_weight
        cr = np.divide(np.sum(total_cap), np.sum(total_weight))
        # print(total_weight)
        # print(total_cap)

        cr_list.append(cr)
        # print('Empirical C.R.:', cr)
    cr_list_highs.append(cr_list)

        # print('Theoretical C.R.:', alpha)
        # print(np.array_equal(packed_idx, packed_items))

In [ ]:
# print(len(packed_items))
# print(packed_items)

In [67]:
cr_means=[]
cr_stds=[]
for i in range(len(cr_list_highs)):
    cr_list = cr_list_highs[i]
    cr_means.append(np.mean(cr_list))
    cr_stds.append(np.std(cr_list))

In [68]:
print(cr_means)

[0.9986233443524949,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9897396703725114,
 0.9767875739170258,
 0.918733372903452]

In [69]:
print(cr_stds)

[0.003225020554787209,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.013004915328016509,
 0.015527663878928517,
 0.05599427823765819]

## Non-infinitesimal Setting (Bonus)

In [74]:
## non-infinitesimal setting

weight_lows=np.linspace(10**9, 10**10, num=10, endpoint=True)

# weight_lows=np.ones(10)
# for i in range(len(weight_lows)):
#     weight_lows[i] = 10 * 10**i
print(weight_lows)

cr_list_highs=[]
for weight_low in weight_lows:
    cr_list=[]
    for itrs in range(100):
        T=100

        # infinitesimal setting
        p_min= 1
        p_max= 100
        # weight_low = 1
        weight_high = 10**10


        values, weights = gen_inputs(T=T, weight_low=weight_low, weight_high=weight_high, p_min=p_min, p_max=p_max)
        # capacities=[max(1, int(np.sum(weights)/2))]
        capacities=[10**10, 10**10]
        # p_max= 10
        # print('Capacities:', capacities)

        total_val, total_cap, alpha, x_ts = get_online_res(values, weights, capacities, p_min, p_max)
        packed_idx=[i for i, e in enumerate(x_ts) if e != 0]

        computed_value, total_weight, packed_items = get_offline_res(values, weights, capacities)
        # cr = total_cap/total_weight
        cr = np.divide(np.sum(total_cap), np.sum(total_weight))
        # print(total_weight)
        # print(total_cap)

        cr_list.append(cr)
        # print('Empirical C.R.:', cr)
    cr_list_highs.append(cr_list)

        # print('Theoretical C.R.:', alpha)
        # print(np.array_equal(packed_idx, packed_items))

[1.e+09 2.e+09 3.e+09 4.e+09 5.e+09 6.e+09 7.e+09 8.e+09 9.e+09 1.e+10]


In [75]:
cr_means=[]
cr_stds=[]
for i in range(len(cr_list_highs)):
    cr_list = cr_list_highs[i]
    cr_means.append(np.mean(cr_list))
    cr_stds.append(np.std(cr_list))

In [76]:
print(cr_means)

[0.8755471107466442,
 0.857823048427452,
 0.8497686559163795,
 0.8306858450779745,
 0.8242073072913898,
 0.8824838278625229,
 0.9134474674332508,
 0.949871436474605,
 0.9810758100970906,
 1.0]

In [77]:
print(cr_stds)

[0.1310723621959483,
 0.131955164118233,
 0.1655979915325303,
 0.1732383532939581,
 0.14852661740967107,
 0.10486455564247682,
 0.07486230248330367,
 0.049110235931654526,
 0.025514082939548696,
 0.0]